In [1]:
import anndata as ad
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import scvi 

/home/albert.baichorov/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


# PCA emb 

In [30]:
cell_embeddings = pd.read_csv('/home/albert.baichorov/ImSysAging/scRNA-seqEmbAge.csv', index_col=0)

In [31]:
cell_types = cell_embeddings['secondary_type'].unique()

res = []
for cell_type in cell_types:
    filtered_data = cell_embeddings[cell_embeddings['secondary_type'] == cell_type]
    res.append([cell_type, filtered_data['age'].mean(), filtered_data['age'].std()])
pd.DataFrame(res, columns=['cell_type', 'mean_age', 'std_age'])

,cell_type,mean_age,std_age
0,CD4_TEM_ANXA1,42.130787,23.581463
1,CD8_TEM_GNLY,41.161710,29.622670
2,B_Memory,30.269392,25.353150
3,NK_CD56_Dim,34.550202,31.036562
4,B_Naive,21.682565,25.916784
5,CD4_TEM_GNLY,56.869298,27.221909
6,CD8_TEM_CMC1,45.426961,28.476375
7,CD4_TCM_AQP3,33.741631,27.333291
8,Monocytes_CD14,30.542219,27.752696
9,CD4_Naive_CCR7,24.242486,26.596497


In [32]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedGroupKFold
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor


In [40]:
def train_model(filtered_data):
    X_train, X_test, y_train, y_test = train_test_split(
        filtered_data.drop(columns=['age']),
        filtered_data['age'],
        test_size=0.3,
        random_state=42
        )

    parameters = {
        'n_estimators': [100,500],
        'max_features': [0.4]
    }
    cv = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=0)
    model = GridSearchCV(RandomForestRegressor(n_jobs=-1),
                         parameters,
                         cv=cv,
                         scoring='r2',
                         verbose=1,
                         n_jobs=-1)
    #model = RandomForestRegressor(n_estimators=500, max_features=0.3, n_jobs=-1)
    model.fit(X_train, y_train, groups=y_train)
    y_pred = model.predict(X_test)

    return r2_score(y_test, y_pred)#, model.best_params_




In [41]:
res = []
for cell_type in cell_types:
    print(cell_type)
    filtered_data = cell_embeddings[cell_embeddings['secondary_type'] == cell_type] \
        .drop(columns=['sampleName', 'primary_type', 'secondary_type'])
    ts = train_model(filtered_data)
    print(f'test_score: {ts}')
    res.append([cell_type, ts])
report = pd.DataFrame(res, columns=['cell_type', 'test r2'])
report

CD4_TEM_ANXA1
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.15620713541128228
CD8_TEM_GNLY
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.23851550717681136
B_Memory
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.11201539112370162
NK_CD56_Dim
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.14028098332087213
B_Naive
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.1385755260127184
CD4_TEM_GNLY
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.13668112524604747
CD8_TEM_CMC1
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.21650109481104918
CD4_TCM_AQP3
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.19521059233184634
Monocytes_CD14
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.062166072933218075
CD4_Naive_CCR7
Fitting 3 folds for each of 2 candidates, totalling 6 fi

/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:994: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


test_score: 0.08779145398752297
CD8_Naive_LEF1
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.23564293924211344
gdT
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.2380327737897544
NK_CD56_Bright
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.0970532001071509
B_Atypical_Memory
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.09777392743399294
Monocytes_CD16
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.03737987793828845
NK_Proliferating
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.1129276294382362
CD4_Treg_FOXP3
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.13188129709156104
pDC
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:994: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


test_score: 0.02100049837831075
CD8_TCM_HAVCR2
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.2779791932379796
B_BCR_GNLY
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:994: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


test_score: 0.2644357810442729
CD8_TEM_ZNF683
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.09299033172575044
mDC
Fitting 3 folds for each of 2 candidates, totalling 6 fits
test_score: 0.12025294481433568
Plasma cell
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:994: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


test_score: 0.1314644590644941


,cell_type,test r2
0,CD4_TEM_ANXA1,0.156207
1,CD8_TEM_GNLY,0.238516
2,B_Memory,0.112015
3,NK_CD56_Dim,0.140281
4,B_Naive,0.138576
5,CD4_TEM_GNLY,0.136681
6,CD8_TEM_CMC1,0.216501
7,CD4_TCM_AQP3,0.195211
8,Monocytes_CD14,0.062166
9,CD4_Naive_CCR7,0.301508


# Normalized Counts 

In [1]:
import anndata as ad
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import scvi 
import numpy as np

import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedGroupKFold
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

/home/albert.baichorov/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [2]:
adata = ad.read_h5ad('/home/albert.baichorov/ImSysAging/scRNA-seq.h5ad')

In [3]:
del adata.layers['RNA_data']

In [4]:
adata

AnnData object with n_obs × n_vars = 538266 × 22091
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.HB', 'percent.RPS', 'S.Score', 'G2M.Score', 'Phase', 'umis_TCR', 'umis_BCR', 'primary_type', 'secondary_type', 'sequential_group', 'eight_group', 'sampleName', 'age'
    uns: 'version'
    layers: 'RNA_counts'

In [ ]:
sc.pp.combat(adata, key="sampleName")


Found 316 genes with zero variance.


In [13]:
def train_model_counts(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.3,
        random_state=42
        )

    parameters = {
        'n_estimators': [100,500],
        'max_features': [0.4]
    }
    cv = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=0)
    model = GridSearchCV(RandomForestRegressor(n_jobs=-1),
                         parameters,
                         cv=cv,
                         scoring='r2',
                         verbose=1,
                         n_jobs=-1)
    #model = RandomForestRegressor(n_estimators=500, max_features=0.3, n_jobs=-1)
    model.fit(X_train, y_train, groups=y_train)
    y_pred = model.predict(X_test)

    return r2_score(y_test, y_pred)#, model.best_params_

In [14]:
res = []

cell_types = adata.obs['secondary_type'].unique()

for cell_type in cell_types:
    print(cell_type)
    X = adata[ adata.obs['secondary_type'] == cell_type, :].layers['RNA_data'].toarray()
    y = adata[ adata.obs['secondary_type'] == 'CD4_TEM_ANXA1', :].obs['age'].astype(float)
    mask = ~np.isnan(X).any(axis=1)
    y = y[mask]
    X = X[mask]
    ts = train_model_counts(X, y)
    print(f'test_score: {ts}')
    res.append([cell_type, ts])
report = pd.DataFrame(res, columns=['cell_type', 'test r2'])
report

CD4_TEM_ANXA1
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:745: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:745: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:745: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
/home/albert.baichoro

ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 377, in fit
    estimator._compute_missing_values_in_feature_mask(
  File "/home/albert.baichorov/.venv/lib/python3.10/site-packages/sklearn/tree/_classes.py", line 222, in _compute_missing_values_in_feature

In [15]:
X.max()

np.float64(1.767903459113713e+308)